Understanding `sympy` through integer scaling for images


### Problem:

Images have different dimensions than displays, and often need to be stretched/shrunk to fit. To avoid distorting the image, we often want to use *integer scaling*. That is, if enlarging a smaller image, distorting it in such a way that a single (square) 1x1 pixel in the smaller image is stretched to a square in the larger image (say 2x2 or 3x3). Then shapes are preserved (though you might need black bars above/to the sides if the aspect ratios differ).

For a larger image, shrinking a square area (say 5x5) to a single (square) pixel, also doesn't distort/squeeze the image.

Doing this in pure python is imprecise, so `sympy`, which allows us to work with ratios such as 16/9 or 10/4 instead of `1.7777777777777777` and `2.5`, allows us to avoid imprecision that comes with floating point. 

In [4]:
from math import ceil, floor

from sympy import Integer, Mul, evaluate, simplify

In [5]:

def scaler(d: Mul, i: Mul):
    """
    Provides optimum integer scaling factor to scale a image resolution (w/h) to a display resolution (w/h)
    while maintaining aspect ratio

    Args:
        d (Mul): Display resolution, eg. 720/576 (both must be +ve integers)
        i (Mul): Image resolution, eg. 640/480 (both must be +ve integers)
    """
    assert type(d) is Mul, "malformed input 1, expected Mul"
    assert type(i) is Mul, "malformed input 2, expected Mul"
    assert not any(x < 0 for x in d.args), "Dimensions must be positive"
    assert not any(x < 0 for x in i.args), "Dimensions must be positive"
    print("Aspect ratios: ", i, " (", i.simplify(), ") ",
          "--> ", d, " (", d.simplify(), ") ", sep='')

    ratio_width = d.args[0] / i.args[0]
    ratio_height = i.args[-1] / d.args[-1]
    if min(ratio_width, ratio_height) > 1:  # image must be scaled UP (in both dimensions display>image)
        ratio = min(ratio_width, ratio_height)
        print("Perfect scaling is", ratio)
        ratio2 = floor(ratio.evalf())
        a = simplify(i.args[0] * ratio2)  # new width
        b = simplify(i.args[-1]**(-1) * ratio2)  # new height
        if ratio != ratio2:
            print("Integer scaling is", ratio2)
        with evaluate(False):
            c = a / b
        print("Best image:", c)
        return 0
    # image must be scaled DOWN (in some dimension image>display)
    elif min(ratio_width, ratio_height) < 1:
        # dimension in which image is (relatively) largest
        ratio = min(ratio_width, ratio_height)
        print("Perfect scaling is", ratio)
        ratio2 = ceil((ratio**(-1)).evalf())**-1
        a = simplify(i.args[0] * ratio2)  # new width
        b = simplify(i.args[-1]**(-1) * ratio2)  # new height
        # if optimal integer scaling returns a noninteger
        while (type(a) is not Integer) or (type(b) is not Integer):
            ratio2 = (ratio2**-1 + 1)**-1
            if a < 1 or b < 1:
                print("Optimal scale not found")
                return -1
            a = simplify(i.args[0] * ratio2)  # new width
            b = simplify(i.args[-1]**(-1) * ratio2)  # new height
        if ratio != ratio2:  # New scale even if only for returning integral dimensions
            print("Integer scaling is", ratio2)
        with evaluate(False):
            c = a / b
        print("Best image:", c)
        return 0
    else:
        print("Already Scaled")
        return 0


# transforming a 720x576 image to QHD (2560x1440)
display_w = Integer(2560)
display_h = Integer(1440)
image_w = Integer(720)
image_h = Integer(576)

with evaluate(False):
    display_res = display_w / display_h
    image_res = image_w / image_h

scaler(display_res, image_res)

Aspect ratios: 720/576 (5/4) --> 2560/1440 (16/9) 
Perfect scaling is 5/2
Integer scaling is 2
Best image: 1440/1152


0

In [7]:
# transforming a UHD (3840x2160) image to 704x480
image_w = Integer(3840)
image_h = Integer(2160)
display_w = Integer(704)
display_h = Integer(480)

with evaluate(False):
    display_res = display_w / display_h
    image_res = image_w / image_h

scaler(display_res, image_res)


Aspect ratios: 3840/2160 (16/9) --> 704/480 (22/15) 
Perfect scaling is 11/60
Integer scaling is 1/6
Best image: 640/360


0

In [8]:
image_w = Integer(3840)
image_h = Integer(2160)
display_w = Integer(1204)
display_h = Integer(5480)

with evaluate(False):
    display_res = display_w / display_h
    image_res = image_w / image_h

scaler(display_res, image_res)


Aspect ratios: 3840/2160 (16/9) --> 1204/5480 (301/1370) 
Perfect scaling is 301/960
Integer scaling is 1/4
Best image: 960/540


0

In [9]:
image_w = Integer(3840)
image_h = Integer(2160)
display_w = Integer(8000)
display_h = Integer(2080)

with evaluate(False):
    display_res = display_w / display_h
    image_res = image_w / image_h

scaler(display_res, image_res)


Aspect ratios: 3840/2160 (16/9) --> 8000/2080 (50/13) 
Perfect scaling is 26/27
Integer scaling is 1/2
Best image: 1920/1080


0